<a href="https://colab.research.google.com/github/DmitryKutsev/cross-document_coreference/blob/main/classifiers/good_eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Импорт**

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


In [2]:
from IPython.display import SVG, display, HTML

In [3]:
from google.colab import drive
drive.mount('./content', force_remount=False)

Mounted at ./content


In [4]:
import spacy
import numpy as np

import gensim
from gensim.models import Word2Vec 
from gensim.models import FastText

from nltk.tokenize import RegexpTokenizer
from nltk import  sent_tokenize
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
from graphviz import Source
from sklearn import tree, svm, linear_model
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier


In [6]:
!ls
import os
for i in os.listdir('content/'):
  print(i)

content  sample_data
.file-revisions-by-id
.shortcut-targets-by-id
MyDrive
.Trash


In [7]:
import csv
import re
from google.colab import files
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

## **Обработка данных**

In [8]:
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/neuroleptic_antidepr2.txt
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/rakpobedim_ru2.txt
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/hv-info_gepatit2.txt

--2021-05-24 07:15:00--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/neuroleptic_antidepr2.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11655409 (11M) [text/plain]
Saving to: ‘neuroleptic_antidepr2.txt’

neuroleptic_antidep 100%[===================>]  11.12M  17.0MB/s    in 0.7s    

2021-05-24 07:15:03 (17.0 MB/s) - ‘neuroleptic_antidepr2.txt’ saved [11655409/11655409]

--2021-05-24 07:15:03--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/rakpobedim_ru2.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443.

In [9]:
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/coref_df_anns60k.csv
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/annotation_table.csv

--2021-05-24 07:15:08--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/coref_df_anns60k.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2021-05-24 07:15:09 ERROR 404: Not Found.

--2021-05-24 07:15:09--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/annotation_table.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2100361 (2.0M) [text/plain]
Saving to: ‘annotation_table.csv’

annotation_table.cs 100%[===================>]   2.00M  8.94MB/s    in 0.2s

In [10]:
handler = open('neuroleptic_antidepr2.txt', 'r', encoding='utf-8')
my_txt = handler.read()
handler.close()

handler = open('hv-info_gepatit2.txt', 'r', encoding='utf-8')
my_txt += handler.read()
handler.close()

handler = open('rakpobedim_ru2.txt', 'r', encoding='utf-8')
my_txt += handler.read()
handler.close()

In [11]:
posts = my_txt.split('----')

## **Обучение векторизатора**

In [12]:
tokenizer = RegexpTokenizer(r'\w+')
sentences_tokenized = [w.lower() for w in posts]


sentences_tokenized = [tokenizer.tokenize(i) for i in sentences_tokenized]
ft = FastText(sentences_tokenized, window=5, min_count=5, workers=4,sg=1)

Обработка битых символов, чтобы pandas мог открыть таблицу.

In [13]:
count = 0
with open('annotation_table.csv') as csvfile:
  with open('annotations.csv', 'w') as write_file:
    writer = csv.writer(write_file, delimiter='\t')
    for new_row in csvfile:
      count +=1
      # print(i.split('\t'))
      if len(new_row.split('\t')) > 19:
        new_row = new_row.split('\t')[:20]
        new_row[-1] = new_row[-1][:-1] # cut \n
      writer.writerow(new_row)
count

27019

In [14]:
# # with open('coref_df_anns60k.csv') as csvfile:
# with open('new_anns60k.csv') as csvfile:
#   reader = csv.reader(csvfile, delimiter='\t')
#   with open('new_df_anns60k.csv', 'w') as write_file:
#     writer = csv.writer(write_file, delimiter='\t')
#     c = 0
#     for row in reader:
#       c +=1
#       new_row = [i if i and not re.match(r'\t+?', i) else 0 for i in row]
#       if len(new_row) > 9:
#         new_row = new_row[:9]
#       else:
#         new_row = [0 for i in range(9)]
#       writer.writerow(new_row)
# c


In [15]:
# !rm new_anns60k.csv.1 new_anns60k.csv.2 new_df_anns60k.csv new_anns60k.csv
!ls
# !rm annotation_table.csv.2 annotation_table.csv.1 annotation_table.csv annotations.csv

annotations.csv       hv-info_gepatit2.txt	 sample_data
annotation_table.csv  neuroleptic_antidepr2.txt
content		      rakpobedim_ru2.txt


In [16]:
# anns_df = pd.read_csv('coref_df_anns60k.csv', delimiter='\t', error_bad_lines=False, header=None)
anns_df = pd.read_csv('annotations.csv', delimiter='\t')
anns_df

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER
0,обсуждаем,0,1,0.0,0.0,0.0,NaN,NaN,NaN,1per,VERB,ROOT,обсуждаем,NaN,NaN,NaN,VERB,0,0.0,plur
1,препарат,0,1,0.0,0.0,1.0,inan,nomn,masc,NaN,NOUN,obj,обсуждаем,NaN,NaN,NaN,VERB,0,0.0,sing
2,рисперидон,0,1,0.0,0.0,2.0,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,2.0,sing
3,рисполепт,0,1,0.0,0.0,3.0,inan,gent,femn,NaN,NOUN,parataxis,рисперидон,anim,nomn,masc,NOUN,1,3.0,plur
4,сперидан,0,1,0.0,0.0,4.0,NaN,NaN,masc,NaN,PRTS,conj,рисполепт,inan,gent,femn,NOUN,1,4.0,sing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27013,и,0,22,68.0,0.0,2385.0,NaN,NaN,NaN,NaN,CONJ,cc,пошлины,inan,nomn,femn,NOUN,0,0.0,NaN
27014,таможенные,0,22,68.0,0.0,2386.0,NaN,nomn,NaN,NaN,ADJF,amod,пошлины,inan,nomn,femn,NOUN,0,0.0,plur
27015,пошлины,0,22,68.0,0.0,2387.0,inan,nomn,femn,NaN,NOUN,nsubj,есть,NaN,NaN,NaN,INFN,0,0.0,plur
27016,как,0,22,68.0,0.0,2388.0,NaN,NaN,NaN,NaN,CONJ,advmod,есть,NaN,NaN,NaN,INFN,0,0.0,NaN


In [17]:
tok_list = []
for token in anns_df.TOKEN:
  try:
    tok_list.append(ft.wv.__getitem__(token.lower()))
  except (KeyError, AttributeError):
    tok_list.append(0)
    # print(token, 'here')
anns_df['TOKEN_VECT'] = tok_list


tok_list = []
for token in anns_df.HEAD:
  try:
    tok_list.append(ft.wv.__getitem__(token.lower()))
  except (KeyError, AttributeError):
    tok_list.append(0)
    # print(token, 'here')
anns_df['HEAD_VECT'] = tok_list

In [18]:
# tok_list = []
# for sent in 
# for token in anns_df.HEAD:
#   try:
#     tok_list.append(ft.wv.__getitem__(token.lower()))
#   except KeyError:
#     tok_list.append(0)
#     print(token, 'here')
# anns_df['TOKEN_VECT'] = tok_list

In [19]:
anns_df = anns_df.loc[(anns_df['TOPIC_NUM'].isin(list(range(30)))) | (anns_df['SENT_NUM'].isin(list(range(10000))))]
anns_df = anns_df.reset_index().drop(['index'], axis=1)

In [20]:
topics = list(set(anns_df.TOPIC_NUM.values))
topics
sent_vect_list = []
# номера предложений топика
for topic_num in topics:

  sents_in_topic = list(set(anns_df.loc[anns_df['TOPIC_NUM'] == topic_num].SENT_NUM))
  topic_len = len(anns_df.loc[anns_df['TOPIC_NUM'] == topic_num])
  for sent_num in sents_in_topic:
    tok_vects = anns_df.loc[(anns_df['TOPIC_NUM'] == topic_num) & (anns_df['SENT_NUM'] == sent_num)].TOKEN_VECT
    try:
      sent_vect = sum(list(tok_vects))/len(tok_vects)
    except ZeroDivisionError:
      sent_vect = 0
      print(anns_df.loc[(anns_df['SENT_NUM'] == sent_num)])
    for i in range(len(tok_vects)):
      sent_vect_list.append(sent_vect)
len(sent_vect_list)
# если не класть нулевых предложений - будет длина датафрейма, как он раньше импортировлся, но поздно.

26497

In [21]:
anns_df['SENT_VECT'] = sent_vect_list

Проверим данные

In [22]:
anns_df.SENT_NUM[18]

1.0

In [23]:
# topics = list(set(df.TOPIC_NUM.values)) 
# for topic_num in topics:
#   # if topic_num == 2:
#   #   break
#   # извлекаем предложения топика
#   sents_in_topic = list(set(df.loc[df['TOPIC_NUM'] == topic_num ].SENT_NUM))
#   for sent_num in sents_in_topic:
#     sent = df.loc[(df['TOPIC_NUM'] == topic_num)  & (df['SENT_NUM'] == sent_num)]

In [24]:
# vvv = 0
# with open('content/MyDrive/coref_fasttext60k_1.csv') as csvfile:
#   with open('new_coref_df_anns60k.csv', 'w') as write_file:
#     writer = csv.writer(write_file, delimiter='\t')
#     for new_row in csvfile:
#       vvv +=1
#       # print(i.split('\t'))
#       if len(new_row.split('\t')) > 9:
#         new_row = new_row.split('\t')[:9]
#       writer.writerow(new_row)
# vvv

In [25]:
# cor_df = pd.read_csv('content/MyDrive/coref_fasttext60k_1.csv', delimiter=' ')
# cor_df

In [26]:
# cosine_similarity([np.matrix(cor_df.TOKEN_VECT[2]).A[0]], [np.matrix(cor_df.TOKEN_VECT[3]).A[0]])[-1][-1]

# **VECTORS HANDLING**

Изменяем значения векторов предложений, главного слова группы, зависимых слов группы, и токена на косинусную дистанцию с семантической осью.

Сначала, изменим строковое представление на массив.

In [27]:
# cor_df.TOKEN_VECT = [np.matrix(i).A[0] for i in cor_df.TOKEN_VECT]
# cor_df.HEAD_VECT = [np.matrix(i).A[0] for i in cor_df.HEAD_VECT]
# cor_df.SENT_VECT = [np.matrix(i).A[0] for i in cor_df.SENT_VECT]
# cor_df.HEAD_CHILDS_VECT = [np.matrix(i).A[0] for i in cor_df.HEAD_CHILDS_VECT]


Добавим размеченные данные, и посмотрим, что получилось.

In [28]:
# cor_df['NER'] = anns_df['NER']
# cor_df['COREFERENCE_CLUSTER'] = anns_df['COREFERENCE_CLUSTER']
# cor_df['ANN_TOKEN'] = anns_df['TOKEN']
# cor_df

In [29]:
# cor_df.loc()

Ограничим датафрейм теми данными, которые были размечены, чтобы была возможность провести оценку системы.

In [30]:
df = anns_df.iloc[:15000,:]

Всего наименований препаратов размечено (это не обязательно антецеденты, любое название препарата).

In [31]:
len(df.loc[df['NER'] == 1])

581

Из них местоимений (прономинальные анафоры).

In [32]:
len(df.loc[df['NER'] == 3])

113

И номинативные анафоры существительные, такие как "препарат" и "лекарство". В основном это они и есть.

In [33]:
len(df.loc[df['NER'] == 2])

215

In [34]:
len(df.loc[df['NER'].isin([1,2,3])])

909

In [35]:
# df['NER'] = [int(i) for i in df['NER']]
# df['COREFERENCE_CLUSTER'] = [int(i) for i in df['COREFERENCE_CLUSTER']]

Посмотрим на данные.

In [36]:
df.loc[df['NER'].isin([1,2,3])]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
2,рисперидон,0,1,0.0,0.0,2.0,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,2.0,sing,"[0.6422002, 0.9203333, 1.2091188, -0.39404687,...","[-0.00036136972, 1.0909841, -0.019530026, 0.56...","[0.17616315, 0.3446933, 0.058193, -0.062634565..."
3,рисполепт,0,1,0.0,0.0,3.0,inan,gent,femn,NaN,NOUN,parataxis,рисперидон,anim,nomn,masc,NOUN,1,3.0,plur,"[0.3091053, 1.0033083, 0.94316226, -0.06092452...","[0.6422002, 0.9203333, 1.2091188, -0.39404687,...","[0.17616315, 0.3446933, 0.058193, -0.062634565..."
4,сперидан,0,1,0.0,0.0,4.0,NaN,NaN,masc,NaN,PRTS,conj,рисполепт,inan,gent,femn,NOUN,1,4.0,sing,"[0.29151055, 0.6506362, 0.6896492, -0.36765492...","[0.3091053, 1.0033083, 0.94316226, -0.06092452...","[0.17616315, 0.3446933, 0.058193, -0.062634565..."
21,лекарством,0,1,1.0,1.0,21.0,inan,ablt,neut,NaN,NOUN,conj,нет,NaN,NaN,NaN,PRED,2,0.0,sing,"[-0.682182, 0.44038662, -0.4116917, 0.12339758...","[0.55707866, -0.009803482, 0.6789394, 0.377175...","[0.17616315, 0.3446933, 0.058193, -0.062634565..."
142,рисполепта,0,1,1.0,13.0,142.0,inan,nomn,femn,NaN,NOUN,nmod,дозировку,inan,accs,femn,NOUN,1,142.0,sing,"[0.3532758, 0.7841546, 0.8411142, -0.10512489,...","[0.41659012, 0.8700146, 0.8149667, 0.005126129...","[0.21825342, 0.39027682, 0.063128404, -0.11102..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14807,абраксан,1,15,18.0,3.0,2215.0,anim,gent,femn,NaN,NOUN,obj,спросите,NaN,NaN,NaN,VERB,1,2215.0,plur,"[0.5964129, 0.89726555, 0.44491175, -0.4539613...","[0.31332228, 0.060981937, -0.2916319, -0.62042...","[0.16565439, 0.37797835, 0.11320429, -0.171806..."
14809,гемцитабином,1,15,18.0,3.0,2217.0,inan,ablt,masc,NaN,NOUN,nmod,абраксан,anim,gent,femn,NOUN,1,2217.0,sing,"[0.084998034, 0.7242142, 0.18191563, 0.1517318...","[0.5964129, 0.89726555, 0.44491175, -0.4539613...","[0.16565439, 0.37797835, 0.11320429, -0.171806..."
14856,фолфиринокс,1,15,18.0,8.0,2264.0,inan,nomn,masc,NaN,NOUN,nmod,схема,inan,nomn,femn,NOUN,1,2264.0,sing,"[0.33518997, 0.71797556, 0.14390625, -0.315288...","[0.17215654, 0.2995089, 0.47286636, -0.5153573...","[0.16565439, 0.37797835, 0.11320429, -0.171806..."
14887,иринотекану,1,15,19.0,0.0,2295.0,inan,datv,masc,NaN,NOUN,nmod,аннотации,inan,nomn,femn,NOUN,1,2295.0,sing,"[0.29551807, 0.78757876, 0.05905184, 0.0468995...","[0.002435874, 0.8253885, 0.04704721, 0.1811256...","[0.16565439, 0.37797835, 0.11320429, -0.171806..."


# **SEMANTIC AXIS**

Реализация метода семантических осей.
Сначала, выбираем первые предложения каждого топика. Посмотрим на них.

In [37]:
axis_df = df.query('POST_NUM == 0 and TOKEN_NUM == 2 and SENT_NUM == 0')
axis_df

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
2,рисперидон,0,1,0.0,0.0,2.0,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,2.0,sing,"[0.6422002, 0.9203333, 1.2091188, -0.39404687,...","[-0.00036136972, 1.0909841, -0.019530026, 0.56...","[0.17616315, 0.3446933, 0.058193, -0.062634565..."
576,зелдокс,0,2,0.0,0.0,2.0,inan,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,2.0,sing,"[0.52026033, 1.042701, 0.27475935, -0.15548646...","[-0.00036136972, 1.0909841, -0.019530026, 0.56...","[-0.20711449, 0.24946558, -0.012944454, -0.155..."
1870,оланзапин,0,3,0.0,0.0,2.0,inan,gent,femn,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,2.0,plur,"[0.51082295, 0.91090816, 0.6259405, -0.3483964...","[-0.00036136972, 1.0909841, -0.019530026, 0.56...","[0.16483341, 0.3735418, 0.10321984, -0.1872682..."
5555,галоперидол,0,4,0.0,0.0,2.0,inan,nomn,masc,NaN,NOUN,obj,обсуждаем,NaN,NaN,NaN,VERB,1,2.0,sing,"[0.36183417, 1.1203966, 0.63945556, -0.4248387...","[-0.74677086, 0.25333044, -0.5015719, -0.44010...","[0.32603085, 0.24580134, -0.1411523, -0.105464..."
6200,тиаприд,0,5,0.0,0.0,2.0,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,3.0,sing,"[0.30627972, 1.0065159, 0.8827934, -0.29866943...","[-0.00036136972, 1.0909841, -0.019530026, 0.56...","[0.18294002, 0.38527223, 0.21451744, -0.169810..."
8450,сероквель,0,6,0.0,0.0,2.0,inan,gent,femn,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,2.0,plur,"[0.43759695, 1.1104311, 0.87404096, 0.07090642...","[-0.00036136972, 1.0909841, -0.019530026, 0.56...","[0.2679471, 0.44669122, 0.058715444, -0.132957..."
8541,день,0,7,0.0,0.0,2.0,inan,accs,masc,NaN,NOUN,ROOT,день,inan,accs,masc,NOUN,0,0.0,sing,"[0.60911465, 0.5871983, -0.17297894, 0.1846828...","[0.60911465, 0.5871983, -0.17297894, 0.1846828...","[0.22502188, 0.27203906, 0.07441485, -0.170355..."
11117,500,0,8,0.0,0.0,2.0,NaN,NaN,NaN,NaN,NaN,nummod,мг,inan,nomn,masc,NOUN,0,0.0,NaN,"[-0.27041537, -0.70974123, 0.31189176, 0.11940...","[-0.02227375, 0.5922974, 0.67596036, -0.103962...","[0.16980284, 0.3700061, 0.1677584, -0.16217725..."
11787,капецитабин,0,10,0.0,0.0,2.0,anim,nomn,masc,NaN,NOUN,ROOT,капецитабин,anim,nomn,masc,NOUN,1,2.0,sing,"[0.024117725, 0.96247023, 0.39689127, -0.31329...","[0.024117725, 0.96247023, 0.39689127, -0.31329...","[0.16117896, 0.3422137, 0.15571809, -0.1424033..."
11806,иресса,0,11,0.0,0.0,2.0,NaN,NaN,NaN,NaN,GRND,nmod,отдам,NaN,NaN,NaN,VERB,1,2.0,NaN,"[0.14457746, 0.277566, 0.42839533, -0.10226413...","[-0.1835394, -0.9023262, -0.55792195, -0.63457...","[0.16117896, 0.3422137, 0.15571809, -0.1424033..."


In [38]:
# sum(df.query('POST_NUM == 0 and TOKEN_NUM == 2 and SENT_NUM == 0').TOKEN_VECT)/len(df.query('POST_NUM == 0 and TOKEN_NUM == 2 and SENT_NUM == 0').TOKEN_VECT)

Суммируем верктора токенов, главных слов, предложения целиком, и зависимых слов. Нормализуем.

In [39]:

token_axe = sum(axis_df.TOKEN_VECT)/len(axis_df.TOKEN_VECT)
head_axe = sum(axis_df.HEAD_VECT)/len(axis_df.HEAD_VECT)
# head_childs_axe = sum(axis_df.HEAD_CHILDS_VECT)/len(axis_df.HEAD_CHILDS_VECT)
sent_axe = sum(axis_df.SENT_VECT)/len(axis_df.SENT_VECT)

In [40]:
df.TOKEN_VECT[710]

0

Функция, которая заменяет столбец с векторами на столбец со значениями косинусной дистанции с соответствующей семантической осью.


In [41]:
def make_similarity(df, col, axe):
  new_col = []
  for ind, vect in enumerate(df[col]):
    try:
      new_col.append(cosine_similarity([axe], [vect])[-1][-1])
    except ValueError:
      new_col.append(0)
  df[col] = new_col



In [42]:
make_similarity(df, 'TOKEN_VECT', token_axe)
make_similarity(df, 'HEAD_VECT', head_axe)
# make_similarity(df, 'HEAD_CHILDS_VECT', head_childs_axe)
make_similarity(df, 'SENT_VECT', sent_axe)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

In [43]:
df.TOKEN[1]

'препарат'

In [44]:
# df['SUM_VECT']= [(df.TOKEN_VECT[i] + df.SENT_VECT[i])/3 for i in range(len(df.SENT_VECT))]

Посмотрим на обработанные данные.

In [45]:
df

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
0,обсуждаем,0,1,0.0,0.0,0.0,NaN,NaN,NaN,1per,VERB,ROOT,обсуждаем,NaN,NaN,NaN,VERB,0,0.0,plur,0.467785,0.492365,0.976057
1,препарат,0,1,0.0,0.0,1.0,inan,nomn,masc,NaN,NOUN,obj,обсуждаем,NaN,NaN,NaN,VERB,0,0.0,sing,0.560020,0.492365,0.976057
2,рисперидон,0,1,0.0,0.0,2.0,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,2.0,sing,0.788884,0.888401,0.976057
3,рисполепт,0,1,0.0,0.0,3.0,inan,gent,femn,NaN,NOUN,parataxis,рисперидон,anim,nomn,masc,NOUN,1,3.0,plur,0.817054,0.552653,0.976057
4,сперидан,0,1,0.0,0.0,4.0,NaN,NaN,masc,NaN,PRTS,conj,рисполепт,inan,gent,femn,NOUN,1,4.0,sing,0.728285,0.625592,0.976057
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,эта,0,16,3.0,0.0,61.0,NaN,nomn,femn,NaN,ADJF,det,вакцина,inan,nomn,femn,NOUN,0,0.0,sing,0.396118,0.469639,0.944387
14996,вакцина,0,16,3.0,0.0,62.0,inan,nomn,femn,NaN,NOUN,nsubj,аукнется,NaN,NaN,NaN,VERB,0,0.0,sing,0.527859,0.597383,0.944387
14997,аукнется,0,16,3.0,0.0,63.0,NaN,NaN,NaN,3per,VERB,csubj,неизвестно,NaN,NaN,NaN,ADVB,0,0.0,sing,0.551375,0.520354,0.944387
14998,на,0,16,3.0,0.0,64.0,NaN,NaN,NaN,NaN,PREP,case,организме,inan,loct,masc,NOUN,0,0.0,NaN,0.440334,0.500613,0.944387


# **ПРАВИЛОВЫЙ АЛГОРИТМ**

In [46]:
# df.SENT_NUM = [int(i) for i in df['SENT_NUM']]
# df.POST_NUM = [int(i) for i in df['POST_NUM']]
# # df.TOKEN_NUM = [float(i) for i in df['TOKEN_NUM']]
df.TOPIC_NUM = [int(i) for i in df['TOPIC_NUM']]
# df.NER = [int(i) for i in df['NER']]

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [47]:
topics = list(set(df.TOPIC_NUM.values))
topics

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

In [48]:
df.loc[df['TOKEN_NUM'] == 344 ]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
344,рисполепта,1,1,2.0,4.0,344.0,inan,nomn,femn,NaN,NOUN,nmod,приеме,inan,loct,masc,NOUN,1,344.0,sing,0.766231,0.500547,0.964356
918,вышло,1,2,6.0,3.0,344.0,NaN,NaN,neut,NaN,VERB,ROOT,вышло,NaN,NaN,neut,VERB,0,0.0,sing,0.519749,0.486272,0.984156
2212,в,0,3,5.0,8.0,344.0,NaN,NaN,NaN,NaN,PREP,case,использовании,inan,loct,neut,NOUN,0,0.0,NaN,0.464867,0.532210,0.974837
5897,и,0,4,6.0,7.0,344.0,NaN,NaN,NaN,NaN,CONJ,cc,выписывались,NaN,NaN,NaN,VERB,0,0.0,NaN,0.432702,0.467088,0.917469
6542,в,0,5,17.0,0.0,344.0,NaN,NaN,NaN,NaN,PREP,case,мозгу,inan,loc2,masc,NOUN,0,0.0,NaN,0.464867,0.427207,0.986065
8883,их,0,7,0.0,32.0,344.0,NaN,accs,NaN,3per,NPRO,obj,принимать,NaN,NaN,NaN,INFN,0,0.0,plur,0.400450,0.482892,0.946690
11513,отвратно,0,9,0.0,39.0,344.0,NaN,NaN,NaN,NaN,ADVB,advmod,перенесла,NaN,NaN,femn,VERB,0,0.0,NaN,0.543650,0.525929,0.990285
12936,будет,0,15,0.0,31.0,344.0,NaN,NaN,NaN,3per,VERB,advcl,проходить,NaN,NaN,NaN,INFN,0,0.0,sing,0.461428,0.591904,0.973172


In [49]:

topic_num = 1
topic_df = df.loc[df['TOPIC_NUM'] == topic_num]
# номера предложений топика
sents_in_topic = list(set(df.loc[df['TOPIC_NUM'] == topic_num ].SENT_NUM))
sents_in_topic
# достать пост
df.loc[(df['TOPIC_NUM'] == 1) & (df['POST_NUM'] == 1)]
# достать конкретное предложение конкретного поста
df.loc[(df['TOPIC_NUM'] == 1) & (df['POST_NUM'] == 1) & (df['SENT_NUM'] == 1)]
# достать токен из предложение конкретного поста
df.loc[(df['TOPIC_NUM'] == 1) & (df['POST_NUM'] == 1) & (df['SENT_NUM'] == 0)]


,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
5,не,0,1,1.0,0.0,5.0,NaN,NaN,NaN,NaN,PRCL,advmod,знаю,NaN,NaN,NaN,VERB,0,0.0,NaN,0.435302,0.487970,0.976057
6,знаю,0,1,1.0,0.0,6.0,NaN,NaN,NaN,1per,VERB,ROOT,знаю,NaN,NaN,NaN,VERB,0,0.0,sing,0.514558,0.487970,0.976057
7,стоило,0,1,1.0,0.0,7.0,NaN,NaN,neut,NaN,VERB,ccomp,знаю,NaN,NaN,NaN,VERB,0,0.0,sing,0.485684,0.487970,0.976057
8,ли,0,1,1.0,0.0,8.0,NaN,NaN,NaN,NaN,CONJ,advmod,стоило,NaN,NaN,neut,VERB,0,0.0,NaN,0.460467,0.451228,0.976057
9,мне,0,1,1.0,0.0,9.0,NaN,datv,NaN,1per,NPRO,iobj,стоило,NaN,NaN,neut,VERB,0,0.0,sing,0.504242,0.451228,0.976057
10,писать,0,1,1.0,0.0,10.0,NaN,NaN,NaN,NaN,INFN,csubj,стоило,NaN,NaN,neut,VERB,0,0.0,NaN,0.460003,0.451228,0.976057


In [50]:
for i in df.loc[(df['TOPIC_NUM'] == 1) & (df['POST_NUM'] == 1) & (df['SENT_NUM'] == 0)].values:
  print(i[-5])

0.0
0.0
0.0
0.0
0.0
0.0


In [51]:
# анафоры
df.loc[(df['TOPIC_NUM'] == 1) & (df['POST_NUM'] == 1) & (df['SENT_NUM'] == 1)
& (df['NER'] > 1)].values[0]

array(['лекарством', '0', 1, 1.0, 1.0, 21.0, 'inan', 'ablt', 'neut', nan,
       'NOUN', 'conj', 'нет', nan, nan, nan, 'PRED', 2, 0.0, 'sing',
       0.4543353021144867, 0.5150907039642334, 0.9760570526123047],
      dtype=object)

In [52]:
# sents_in_topic 
# topic_num
# df = df.reset_index()

In [54]:
# post = df.loc[(df['TOPIC_NUM'] == topic_num) \
#               & (df['POST_NUM'] == post_num) \
#               & (df['SENT_NUM'] <= sent_num)]

# current_antecedents = post.loc[(sent['NER'] == 1) ]

In [55]:
# knn_clf = KNeighborsClassifier(n_neighbors=10, weights='distance',algorithm='ball_tree',leaf_size=5, metric='minkowski', p=1).fit(X, y)
# knn_clf.score(X_test, y_test)

In [56]:
def rule_eval(my_X, my_y):
  
  tp = []
  fp = []
  tn = []
  fn = []

  for i in range(len(my_X)):
    if my_X[i] != 0:
      if my_X[i] == my_y[i]:
        tp.append(i)
      else:
        fp.append(i)
    # берем только те синглтоны, которые относятся к номинативным или прономинальным группам.
    elif my_X[i] == 0:
      if my_X[i] == my_y[i]:
        tn.append(i)
      else:
        fn.append(i)
  try:
    P = len(tp)/(len(tp) + len(fp))
  except ZeroDivisionError:
    P = 0
  try:
    R = len(tp)/(len(tp) + len(fn))
  except ZeroDivisionError:
    R = 0
  try:
    F = 2*P*R/(P+R)
  except ZeroDivisionError:
    F = 0

  print(len(tp), ' true positive')
  print(len(fp), ' false positive')
  print(len(tn), ' true negative')
  print(len(fn), ' false negative')
  print('---------')
  print(P,' presicion')
  print(R, ' recall')
  print(F, ' f-score')


In [57]:
# def anaphore_in_sent(sent_num, topic_num) это потом
# изначально отталкиваемся от номеров топиков, итерация по ним
# topics = list(set(df.TOPIC_NUM.values)) это выше
ann_clusters = []
my_clusters = []
post_num = 0
for topic_num in topics:
  # print(topics, 'topics')
  # if post_num == 3:
  #   break
  # извлекаем посты топика
  posts_in_topic = list(set(df.loc[df['TOPIC_NUM'] == topic_num].POST_NUM))
  # sents_in_topic = list(set(df.loc[df['TOPIC_NUM'] == topic_num].SENT_NUM))


  for post_num in posts_in_topic:
    sents_in_post = list(set(df.loc[(df['TOPIC_NUM'] == topic_num) \
                                    & (df['POST_NUM'] == post_num)].SENT_NUM))
    # if post_num == 3:
    #   break

    for sent_num in sents_in_post:

      sent = df.loc[(df['TOPIC_NUM'] == topic_num)  & (df['SENT_NUM'] == sent_num) \
                    & (df['POST_NUM'] == post_num)]
      # собираем анафоры и антецеденты
      current_anaphors = sent.loc[(sent['NER'] > 1)]
      current_antecedents = sent.loc[(sent['NER'] == 1)]

      # если нет анафоров, продолжать не имеет смысла
      
      if not list(current_anaphors.TOKEN):
        continue

      # если в предложении есть один антецедент, самый простой случай
      if len(list(current_antecedents.TOKEN)) == 1:
        for anfr in current_anaphors.values:
          # проверяем сходимость по числу
          if anfr[-4] == current_antecedents.NUMBER.values[0]:
            # добавляем в список размеченный класс
            ann_clusters.append(anfr[-5])
            # и класс который определило правило(антецедента из предложения)
            my_clusters.append(current_antecedents.TOKEN_NUM.values[0])
          else:
            # в противном случае присваиваем нулевой класс
            ann_clusters.append(anfr[-5])
            # и класс, который определило правило(антецедента из предложения)
            my_clusters.append(0.0)

      # если больше одного!!
          
      # если в предложении его нет =(
      if len(list(current_antecedents.TOKEN)) == 0:
        # ищем в посте в предложениях выше
        post = df.loc[(df['TOPIC_NUM'] == topic_num) \
                      & (df['POST_NUM'] == post_num) \
                      & (df['SENT_NUM'] <= sent_num)]

        current_antecedents = post.loc[(post['NER'] == 1) ]

        # если его нету в посте, выбираем первый же антецедент из предыдущих постов
        # топика(сверху вниз)
        if not list(current_antecedents.TOKEN):
          topic = df.loc[(df['TOPIC_NUM'] == topic_num) \
                        & (df['POST_NUM'] <= post_num) \
                        & (df['SENT_NUM'] <= sent_num)]
          current_antecedents = topic.loc[(topic['NER'] == 1) ]

          # это можно убрать в отдельную функцию
          for anfr in current_anaphors.values:
            # проверяем сходимость по числу
            try:
              if anfr[-4] == current_antecedents.NUMBER.values[0]:
                # добавляем в список размеченный класс
                ann_clusters.append(anfr[-5])
                # и класс который определило правило(антецедента из предложения)
                my_clusters.append(current_antecedents.TOKEN_NUM.values[0])
                # print(ann_clusters, my_clusters, 'am I here?')
              else:
                # в противном случае присваиваем нулевой класс
                ann_clusters.append(anfr[-5])
                # и класс, который определило правило(антецедента из предложения)
                my_clusters.append(0.0)
            # потом разберусь
            except IndexError:
              my_clusters.append(0.0)
              ann_clusters.append(anfr[-5])
              

        # если антецедент есть в посте раньше, выбираем первый с конца [::-1]
        else:
          # это можно убрать в отдельную функцию
          for anfr in current_anaphors.values:
            # проверяем сходимость по числу
            if anfr[-4] == current_antecedents.NUMBER.values[-1]:
              # добавляем в список размеченный класс
              ann_clusters.append(anfr[-5])
              # и класс который определило правило(антецедента из предложения)
              my_clusters.append(current_antecedents.TOKEN_NUM.values[-1])

            else:
              # в противном случае присваиваем нулевой класс
              ann_clusters.append(anfr[-5])
              # и класс, который определило правило(антецедента из предложения)
              my_clusters.append(0.0)






      # print(list(current_anaphors.TOKEN), list(current_anaphors.TOKEN), list(current_anaphors.COREFERENCE_CLUSTER) )
      # print(list(current_antecedents.TOKEN), list(current_antecedents.TOKEN), list(current_antecedents.COREFERENCE_CLUSTER) )
print(my_clusters)
print('---')
print(ann_clusters)
print('---')





[2.0, 0.0, 0.0, 344.0, 2.0, 2.0, 2.0, 2.0, 540.0, 11.0, 11.0, 46.0, 108.0, 108.0, 182.0, 182.0, 182.0, 182.0, 375.0, 434.0, 461.0, 658.0, 658.0, 693.0, 755.0, 755.0, 829.0, 829.0, 829.0, 829.0, 1022.0, 1081.0, 1081.0, 1142.0, 26.0, 0.0, 0.0, 0.0, 183.0, 0.0, 0.0, 0.0, 2.0, 2.0, 830.0, 0.0, 0.0, 987.0, 0.0, 0.0, 0.0, 1750.0, 1750.0, 1844.0, 0.0, 2031.0, 2053.0, 0.0, 2058.0, 2156.0, 0.0, 2344.0, 2.0, 0.0, 2711.0, 2711.0, 2766.0, 2766.0, 2766.0, 0.0, 2.0, 0.0, 3125.0, 3125.0, 3270.0, 3350.0, 3350.0, 0.0, 0.0, 0.0, 3619.0, 0.0, 2.0, 5.0, 5.0, 2.0, 114.0, 0.0, 174.0, 0.0, 0.0, 0.0, 287.0, 0.0, 287.0, 287.0, 365.0, 365.0, 0.0, 0.0, 449.0, 0.0, 449.0, 449.0, 527.0, 527.0, 0.0, 0.0, 0.0, 0.0, 0.0, 27.0, 41.0, 41.0, 41.0, 2.0, 2.0, 135.0, 135.0, 0.0, 256.0, 270.0, 270.0, 270.0, 2.0, 2.0, 2.0, 482.0, 482.0, 521.0, 521.0, 521.0, 521.0, 0.0, 0.0, 0.0, 0.0, 786.0, 805.0, 805.0, 870.0, 0.0, 965.0, 965.0, 2.0, 2.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 1455.0, 1516.0, 1609.0, 1657.0, 1722

In [58]:
print(len(my_clusters), len(ann_clusters))

273 273


In [59]:
range(len(my_clusters)), range(len(ann_clusters))

(range(0, 273), range(0, 273))

In [60]:
rule_eval(my_clusters, ann_clusters)

101  true positive
57  false positive
65  true negative
50  false negative
---------
0.6392405063291139  presicion
0.6688741721854304  recall
0.6537216828478964  f-score


In [61]:
df.loc[df['SENT_NUM'] != 0.0]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
11,у,0,1,1.0,1.0,11.0,NaN,NaN,NaN,NaN,PREP,case,меня,NaN,accs,NaN,NPRO,0,0.0,NaN,0.481519,0.420024,0.976057
12,меня,0,1,1.0,1.0,12.0,NaN,accs,NaN,1per,NPRO,obl,нет,NaN,NaN,NaN,PRED,0,0.0,sing,0.482882,0.515091,0.976057
13,практически,0,1,1.0,1.0,13.0,NaN,NaN,NaN,NaN,ADVB,advmod,нет,NaN,NaN,NaN,PRED,0,0.0,NaN,0.521414,0.515091,0.976057
14,нет,0,1,1.0,1.0,14.0,NaN,NaN,NaN,NaN,PRED,ROOT,нет,NaN,NaN,NaN,PRED,0,0.0,NaN,0.474732,0.515091,0.976057
15,никаких,0,1,1.0,1.0,15.0,NaN,gent,NaN,NaN,ADJF,det,проблем,inan,gent,femn,NOUN,0,0.0,plur,0.440604,0.549104,0.976057
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14972,только,0,16,1.0,5.0,38.0,NaN,NaN,NaN,NaN,ADVB,advmod,температура,inan,nomn,femn,NOUN,0,0.0,NaN,0.440661,0.348221,0.944387
14973,температура,0,16,1.0,5.0,39.0,inan,nomn,femn,NaN,NOUN,nsubj,была,NaN,NaN,femn,VERB,0,0.0,sing,0.401526,0.440228,0.944387
14974,после,0,16,1.0,5.0,40.0,NaN,NaN,NaN,NaN,PREP,case,прививки,inan,gent,femn,NOUN,0,0.0,NaN,0.439660,0.534284,0.944387
14975,второй,0,16,1.0,5.0,41.0,NaN,gent,femn,NaN,ADJF,amod,прививки,inan,gent,femn,NOUN,0,0.0,sing,0.525459,0.534284,0.944387


In [62]:
for i in range(len(df)):
  current_raw = df.iloc[[i]].loc[df['NER'] != 0.0]
  print(current_raw)
  if i == 3:
    break

Empty DataFrame
Columns: [TOKEN, IS_ANSWER, TOPIC_NUM, POST_NUM, SENT_NUM, TOKEN_NUM, ANIMACY, CASE, GENDER, PERSON, POS, DEPENDENCY, HEAD, HEAD_ANIMACY, HEAD_CASE, HEAD_GENDER, HEAD_POS, NER, COREFERENCE_CLUSTER, NUMBER, TOKEN_VECT, HEAD_VECT, SENT_VECT]
Index: []
Empty DataFrame
Columns: [TOKEN, IS_ANSWER, TOPIC_NUM, POST_NUM, SENT_NUM, TOKEN_NUM, ANIMACY, CASE, GENDER, PERSON, POS, DEPENDENCY, HEAD, HEAD_ANIMACY, HEAD_CASE, HEAD_GENDER, HEAD_POS, NER, COREFERENCE_CLUSTER, NUMBER, TOKEN_VECT, HEAD_VECT, SENT_VECT]
Index: []
        TOKEN IS_ANSWER  TOPIC_NUM  ...  TOKEN_VECT  HEAD_VECT  SENT_VECT
2  рисперидон         0          1  ...    0.788884   0.888401   0.976057

[1 rows x 23 columns]
       TOKEN IS_ANSWER  TOPIC_NUM  ...  TOKEN_VECT  HEAD_VECT  SENT_VECT
3  рисполепт         0          1  ...    0.817054   0.552653   0.976057

[1 rows x 23 columns]


# **МОДЕЛИ.**

Факторизуем троковые данные, чтобы вместо строковых представлений классов были численные.

In [63]:
for_factorize = 'ANIMACY	CASE	GENDER	PERSON	POS	DEPENDENCY	HEAD	HEAD_ANIMACY	HEAD_CASE	HEAD_GENDER	HEAD_POS'.split()
for_factorize

['ANIMACY',
 'CASE',
 'GENDER',
 'PERSON',
 'POS',
 'DEPENDENCY',
 'HEAD',
 'HEAD_ANIMACY',
 'HEAD_CASE',
 'HEAD_GENDER',
 'HEAD_POS']

In [64]:
_, lbl = pd.factorize(df.ANIMACY)
print(lbl, )
_

Index(['inan', 'anim'], dtype='object')


array([-1,  0,  1, ..., -1, -1,  0])

Делаем словарь с маппингом, чтобы можно было достать нужное значение по индексу. В значении -1 хранится None

In [65]:
mapping_dict = {}
for col_name in for_factorize:
  col, lbl = pd.factorize(df[col_name])
  df[col_name] = col
  mapping_dict[col_name] = lbl
mapping_dict  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


{'ANIMACY': Index(['inan', 'anim'], dtype='object'),
 'CASE': Index(['nomn', 'gent', 'datv', 'accs', 'loct', 'ablt', 'loc2', 'voct', 'gen2'], dtype='object'),
 'DEPENDENCY': Index(['ROOT', 'obj', 'appos', 'parataxis', 'conj', 'advmod', 'ccomp', 'iobj',
        'csubj', 'case', 'obl', 'det', 'nsubj', 'cc', 'discourse', 'fixed',
        'mark', 'amod', 'advcl', 'xcomp', 'nummod', 'nmod', 'nsubj:pass',
        'nummod:gov', 'cop', 'acl:relcl', 'acl', 'aux', 'flat:foreign', 'punct',
        'obl:agent', 'flat', 'aux:pass', 'orphan', 'expl', 'csubj:pass'],
       dtype='object'),
 'GENDER': Index(['masc', 'femn', 'neut'], dtype='object'),
 'HEAD': Index(['обсуждаем', 'препарат', 'рисперидон', 'рисполепт', 'знаю', 'стоило',
        'меня', 'нет', 'проблем', 'лекарством',
        ...
        'привился', 'прививки', 'решать', 'неизвестно', 'быстро', 'проводились',
        'испытания', 'аукнется', 'вакцина', 'организме'],
       dtype='object', length=2514),
 'HEAD_ANIMACY': Index(['inan', 'ani

In [66]:

# labels_dict = {}
# test_labels_dict = {}

# for i in X.columns:
#   X[i], lbl = pd.factorize(X[i])
#   labels_dict[i] = lbl

# for i in X_test.columns:
#   X_test[i], lbl = pd.factorize(X_test[i])
#   test_labels_dict[i] = lbl 

# y = cor_df['COREFERENCE_CLUSTER']
# y, y_label = pd.factorize(y)

# y_test = cor_df_test['COREFERENCE_CLUSTER']
# y_test, y_test_label = pd.factorize(y_test)

In [67]:
df.loc[(df['NER'] == 1.0) | (df['NER'] == 2.0 )]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
2,рисперидон,0,1,0.0,0.0,2.0,1,0,0,-1,1,2,1,0,0,0,1,1,2.0,sing,0.788884,0.888401,0.976057
3,рисполепт,0,1,0.0,0.0,3.0,0,1,1,-1,1,3,2,1,0,0,1,1,3.0,plur,0.817054,0.552653,0.976057
4,сперидан,0,1,0.0,0.0,4.0,-1,-1,0,-1,2,4,3,0,1,1,1,1,4.0,sing,0.728285,0.625592,0.976057
21,лекарством,0,1,1.0,1.0,21.0,0,5,2,-1,1,4,7,-1,-1,-1,3,2,0.0,sing,0.454335,0.515091,0.976057
142,рисполепта,0,1,1.0,13.0,142.0,0,0,1,-1,1,21,69,0,2,1,1,1,142.0,sing,0.766231,0.481858,0.962897
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14759,hifu,1,15,17.0,12.0,2167.0,-1,-1,-1,-1,-1,1,1903,-1,-1,-1,0,1,2167.0,NaN,0.535461,0.508761,0.947723
14807,абраксан,1,15,18.0,3.0,2215.0,1,1,1,-1,1,1,2480,-1,-1,-1,0,1,2215.0,plur,0.689645,0.408740,0.944387
14809,гемцитабином,1,15,18.0,3.0,2217.0,0,5,0,-1,1,21,2470,1,1,1,1,1,2217.0,sing,0.590692,0.514184,0.944387
14856,фолфиринокс,1,15,18.0,8.0,2264.0,0,0,0,-1,1,21,1910,0,0,1,1,1,2264.0,sing,0.740064,0.465693,0.944387


Разделим датасет на тестовый и тренировочный. for_train, for_test 

In [68]:
# train, test = train_test_split(df.loc[(df['NER'] == 1.0) | (df['NER'] == 2.0 ) | (df['NER'] == 3.0 )], test_size=0.2)
# train, test = train_test_split(df, test_size=0.2)
train, test = train_test_split(df.loc[(df['NER'] == 2.0 ) | (df['NER'] == 3.0 )], test_size=0.2)

In [69]:
train = train.reset_index()

In [70]:
# train = train.iloc[:,:24].drop(['TOKEN', 'level_0', 'index'], axis=1)
# train = train.drop(['TOKEN', 'index'], axis=1)

In [71]:
test = test.reset_index()

In [72]:
test.head(5)

,index,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
0,7682,он,1,5,41.0,3.0,1484.0,-1,0,0,1,5,12,1427,-1,-1,0,0,3,1455.0,sing,0.485894,0.556755,0.984288
1,11469,обезболивающие,0,9,0.0,34.0,300.0,-1,0,-1,-1,14,26,1557,0,2,1,1,2,301.0,plur,0.612681,0.480567,0.990285
2,5889,препарат,0,4,6.0,6.0,336.0,0,0,0,-1,1,1,1158,-1,-1,-1,0,2,287.0,sing,0.560020,0.483775,0.888895
3,9691,препараты,1,7,5.0,0.0,1152.0,0,0,0,-1,1,12,619,-1,-1,-1,13,2,0.0,plur,0.494250,0.328365,0.968499
4,6254,препарата,0,5,3.0,1.0,56.0,0,1,0,-1,1,21,1264,0,1,2,1,2,41.0,sing,0.499182,0.577588,0.986065


In [73]:
# test = test.iloc[:,:24].drop(['TOKEN', 'index'], axis=1)
# test = test.drop(['TOKEN', 'index'], axis=1)

In [74]:
# test.loc[test['NER'] != 0]

In [75]:
# ['TOPIC_NUM',	'POST_NUM',	'TOKEN_NUM',	'GENDER',	'PERSON', 'TOKEN', 'TOKEN_VECT', 'IS_ANSWER', 'SENT_NUM', 'HEAD_VECT', 'HEAD_CHILDS_VECT', 
#  'index', 'HEAD_ANIMACY',	'HEAD_CASE',	'HEAD_GENDER', 'ANIMACY',	'CASE',	'POS',	'DEPENDENCY',	'HEAD',	'HEAD_POS']

In [76]:
# best set

# X, y, X_test, y_test = train.drop(['TOKEN',  'HEAD_VECT', 'POST_NUM', 'IS_ANSWER', 'PERSON', 'HEAD_CHILDS_VECT', 'TOPIC_NUM', 'CASE', 'SENT_NUM', 'DEPENDENCY',  'index', 'HEAD_ANIMACY',	'HEAD_CASE',	'HEAD_GENDER', 'ANIMACY',				'HEAD',	'HEAD_POS'], axis=1).iloc[:,:18], \
# train.COREFERENCE_CLUSTER, test.drop(['TOKEN',  'HEAD_VECT', 'POST_NUM', 'IS_ANSWER', 'PERSON', 'HEAD_CHILDS_VECT', 'TOPIC_NUM', 'CASE', 'SENT_NUM', 'DEPENDENCY',    'index', 'HEAD_ANIMACY',	'HEAD_CASE',	'HEAD_GENDER', 'ANIMACY',			'HEAD',	'HEAD_POS'], axis=1).iloc[:,:18], test.COREFERENCE_CLUSTER

Делаем наборы для обучения моделей.

In [77]:
X, y, X_test, y_test = train.drop(['TOKEN','GENDER','NUMBER', 'POS', 'HEAD_VECT', 'POST_NUM', 'IS_ANSWER', 'PERSON', 'TOPIC_NUM', 'CASE', 'SENT_NUM', 'DEPENDENCY',  'index', 'HEAD_ANIMACY',	'HEAD_CASE',	'HEAD_GENDER', 'ANIMACY',				'HEAD',	'HEAD_POS'], axis=1).iloc[:,:18], \
train.COREFERENCE_CLUSTER, test.drop(['TOKEN','GENDER', 'NUMBER', 'POS', 'HEAD_VECT', 'POST_NUM', 'IS_ANSWER', 'PERSON', 'TOPIC_NUM', 'CASE', 'SENT_NUM', 'DEPENDENCY',    'index', 'HEAD_ANIMACY',	'HEAD_CASE',	'HEAD_GENDER', 'ANIMACY',			'HEAD',	'HEAD_POS'], axis=1).iloc[:,:18], test.COREFERENCE_CLUSTER

In [78]:
# X, y, X_test, y_test = train.drop(['TOKEN', 'index'], axis=1).iloc[:,:24], train.COREFERENCE_CLUSTER, test.drop(['TOKEN', 'index'], axis=1).iloc[:,:24], test.COREFERENCE_CLUSTER

In [79]:
X.NER = [float(i) for i in X.NER]
X_test.NER = [float(i) for i in X_test.NER]


In [80]:
y

0      2058.0
1       449.0
2         0.0
3       658.0
4         0.0
        ...  
257       0.0
258     564.0
259      58.0
260     135.0
261     806.0
Name: COREFERENCE_CLUSTER, Length: 262, dtype: float64

Обучаем модели.

In [81]:
clf = tree.DecisionTreeClassifier(max_depth=2000, )
clf.fit(X, y)
clf.score(X_test, y_test)

0.696969696969697

In [82]:
clf_entropy = tree.DecisionTreeClassifier(criterion='entropy', max_depth=2000)
clf_entropy.fit(X, y)
clf_entropy.score(X_test, y_test)

0.6818181818181818

In [83]:
randf_clf = RandomForestClassifier(max_depth=3000, n_estimators=600, bootstrap=True, )
randf_clf.fit(X, y)
randf_clf.score(X_test, y_test)

0.7121212121212122

In [84]:
regr = linear_model.LinearRegression()
regr.fit(X, y)
regr.score(X_test, y_test)

1.0

In [85]:
svm_clf = svm.SVC(decision_function_shape='ovo', cache_size=500)
svm_clf.fit(X, y)
svm_clf.score(X_test, y_test)

0.3787878787878788

In [86]:

knn_clf = KNeighborsClassifier(n_neighbors=10, weights='distance',algorithm='ball_tree',leaf_size=5, metric='minkowski', p=1).fit(X, y)
knn_clf.score(X_test, y_test)

0.696969696969697

In [87]:
# from sklearn.naive_bayes import MultinomialNB
# naive_clf = MultinomialNB().fit(X, y)


# **ОЦЕНКА МОДЕЛЕЙ**

In [88]:
def my_eval(X_predicted, y):
  
  tp = []
  fp = []
  tn = []
  fn = []

  for i in range(len(X_predicted)):
    if X_predicted[i] != 0.0:
      if X_predicted[i] == y_test[i]:
        tp.append(i)
      else:
        fp.append(i)
    # берем только те синглтоны, которые относятся к номинативным или прономинальным группам.
    elif X_predicted[i] == 0.0 and test.iloc[:,22:23].iloc[[i]].values[0][0] in (2.0, 3.0):
      if X_predicted[i] == y_test[i]:
        tn.append(i)
      else:
        fn.append(i)
  try:
    P = len(tp)/(len(tp) + len(fp))
  except ZeroDivisionError:
    P = 0
  try:
    R = len(tp)/(len(tp) + len(fn))
  except ZeroDivisionError:
    R = 0
  try:
    F = 2*P*R/(P+R)
  except ZeroDivisionError:
    F = 0

  print(len(tp), ' true positive')
  print(len(fp), ' false positive')
  print(len(tn), ' true negative')
  print(len(fn), ' false negative')
  print('---------')
  print(P,' presicion')
  print(R, ' recall')
  print(F, ' f-score')



In [89]:
pred = clf.predict(X_test)
pred_randf = randf_clf.predict(X_test)
pred_entropy = clf_entropy.predict(X_test)
pred_regr = regr.predict(X_test)
pred_svm = svm_clf.predict(X_test)
pred_knn = knn_clf.predict(X_test)
# naive_pred = naive_clf.predict(X_test)

In [90]:
# my_eval(list(naive_pred), list(y_test))

In [91]:
my_eval(list(pred_knn), list(y_test))

25  true positive
18  false positive
0  true negative
0  false negative
---------
0.5813953488372093  presicion
1.0  recall
0.7352941176470588  f-score


In [92]:
my_eval(list(pred_entropy), list(y_test))

23  true positive
21  false positive
0  true negative
0  false negative
---------
0.5227272727272727  presicion
1.0  recall
0.6865671641791045  f-score


In [93]:
my_eval(list(pred), list(y_test))

24  true positive
20  false positive
0  true negative
0  false negative
---------
0.5454545454545454  presicion
1.0  recall
0.7058823529411764  f-score


In [94]:
my_eval(list(pred_randf), list(y_test))

25  true positive
19  false positive
0  true negative
0  false negative
---------
0.5681818181818182  presicion
1.0  recall
0.7246376811594203  f-score


In [95]:
my_eval(list(pred_regr), list(y_test))

13  true positive
53  false positive
0  true negative
0  false negative
---------
0.19696969696969696  presicion
1.0  recall
0.3291139240506329  f-score


In [96]:
my_eval(list(pred_svm), list(y_test))

3  true positive
18  false positive
0  true negative
0  false negative
---------
0.14285714285714285  presicion
1.0  recall
0.25  f-score
